# 申万行业动量信号模型演示

本演示展示了如何使用申万行业日收益率数据构建动量信号模型，预测市场趋势变化，并通过SHAP监控因子表现。

## 项目结构
- **data**: 数据处理模块（包含申万行业数据处理器）
- **factors**: 动量因子计算模块
- **model**: 市场趋势预测模型
- **optimization**: 超参数优化模块
- **explanation**: 模型解释模块
- **evaluation**: 模型评估模块

## 数据流程
1. 加载申万行业日收益率数据
2. 将行业数据转换为模型可接受格式
3. 计算动量因子
4. 训练预测模型
5. 优化超参数
6. 评估模型性能
7. 解释模型结果

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 导入项目模块
from data.sw_industry_data_processor import load_sw_industry_data, convert_sw_data_for_model
from data.data_generator import load_real_data
from factors.momentum_factors import calculate_momentum_factors
from model.predictor import MarketTrendPredictor
from optimization.hyperparameter_tuning import optimize_hyperparameters
from explanation.model_explanation import create_shap_explainer, calculate_shap_values, analyze_factor_importance
from evaluation.metrics import calculate_performance_metrics

print('模块导入成功')

## 1. 加载申万行业数据

首先加载申万行业日收益率数据，并查看数据的基本信息。

In [ ]:
# 使用示例数据或真实数据
# 这里我们使用示例数据
sw_data_path = 'sample_sw_data.csv'

# 加载申万行业数据
try:
    sw_raw_data = load_sw_industry_data(sw_data_path)
    print('✅ 申万行业数据加载成功')
except:
    # 如果文件不存在，使用示例数据
    print('⚠️  使用内置示例数据')
    sw_raw_data = pd.DataFrame({
        'date': pd.date_range('2020-01-01', periods=100, freq='D'),
        'industry_name': ['银行', '保险', '证券', '房地产', '建筑装饰'] * 20,
        'return': np.random.normal(0.001, 0.02, 100)
    })

print(f'数据记录数: {len(sw_raw_data)}')
print(f'行业数量: {sw_raw_data["industry_name"].nunique()}')
print(f'时间范围: {sw_raw_data["date"].min()} 到 {sw_raw_data["date"].max()}')

# 显示部分数据
sw_raw_data.head(10)

### 1.1 数据可视化

查看不同行业的收益率分布

In [ ]:
# 行业收益率分布
plt.figure(figsize=(12, 6))

# 箱线图
plt.subplot(1, 2, 1)
sw_raw_data.boxplot(column='return', by='industry_name', ax=plt.gca())
plt.title('各行业收益率分布')
plt.ylabel('日收益率')
plt.xticks(rotation=45)

# 时间序列图
plt.subplot(1, 2, 2)
for industry in sw_raw_data['industry_name'].unique()[:5]:
    industry_data = sw_raw_data[sw_raw_data['industry_name'] == industry]
    plt.plot(industry_data['date'], industry_data['return'], label=industry, alpha=0.7)

plt.title('各行业收益率时间序列')
plt.xlabel('日期')
plt.ylabel('日收益率')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 2. 数据转换

将申万行业数据转换为模型可接受的格式

In [ ]:
# 转换数据格式
converted_data = convert_sw_data_for_model(sw_raw_data)

print('✅ 数据转换完成')
print(f'转换后记录数: {len(converted_data)}')
print(f'股票代码(行业映射): {converted_data["stock_code"].unique()}')

# 显示转换后的数据
converted_data.head(10)

## 3. 动量因子计算

计算短、中、长期动量因子

In [ ]:
# 计算动量因子
factor_data = calculate_momentum_factors(converted_data)

print('✅ 动量因子计算完成')
print(f'因子数据记录数: {len(factor_data)}')
print('因子列:', [col for col in factor_data.columns if 'momentum' in col])

# 显示因子统计信息
factor_cols = ['short_momentum', 'medium_momentum', 'long_momentum']
factor_stats = factor_data[factor_cols].describe()
factor_stats

### 3.1 因子相关性分析

In [ ]:
# 计算因子相关性
correlation_matrix = factor_data[factor_cols].corr()

# 绘制热力图
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('动量因子相关性矩阵')
plt.show()

print('因子相关性:')
print(correlation_matrix)

## 4. 模型训练与预测

使用随机森林回归模型训练市场趋势预测器

In [ ]:
# 创建预测器
predictor = MarketTrendPredictor()

# 准备特征和目标变量
features = predictor.prepare_features(factor_data)
target = predictor.prepare_target(factor_data)

print(f'特征数据形状: {features.shape}')
print(f'目标变量形状: {target.shape}')
print(f'特征列: {list(features.columns)}')

# 训练模型
predictor.train(features, target)
print('✅ 模型训练完成')

# 进行预测
predictions = predictor.predict(features)
print(f'预测结果形状: {predictions.shape}')

### 4.1 预测结果可视化

In [ ]:
# 创建预测结果DataFrame
results_df = factor_data.copy()
results_df['predictions'] = predictions

# 绘制预测vs实际值
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(results_df['return'], results_df['predictions'], alpha=0.5)
plt.xlabel('实际收益率')
plt.ylabel('预测收益率')
plt.title('预测vs实际值散点图')
plt.plot([results_df['return'].min(), results_df['return'].max()], 
         [results_df['return'].min(), results_df['return'].max()], 'r--', lw=2)

plt.subplot(1, 2, 2)
sample_data = results_df.sample(n=min(100, len(results_df)))
plt.plot(sample_data['date'], sample_data['return'], 'b-', label='实际', alpha=0.7)
plt.plot(sample_data['date'], sample_data['predictions'], 'r--', label='预测', alpha=0.7)
plt.xlabel('日期')
plt.ylabel('收益率')
plt.title('时间序列预测对比')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 5. 模型评估

评估模型性能，包括MSE、RMSE和信息比率(IR)

In [ ]:
# 评估模型
evaluation_metrics = predictor.evaluate(features, target)

print('📊 模型评估指标:')
for metric, value in evaluation_metrics.items():
    print(f'  {metric}: {value:.6f}')

# 计算预测收益率的综合性能指标
performance_metrics = calculate_performance_metrics(pd.Series(predictions))

print('
📈 预测收益率综合性能指标:')
for metric, value in performance_metrics.items():
    print(f'  {metric}: {value:.6f}')

## 6. 超参数优化

使用Optuna进行超参数优化，以提高模型性能

In [ ]:
# 执行超参数优化
print('🎯 开始超参数优化...')
optimization_result = optimize_hyperparameters(features, target, n_trials=20)

print('✅ 超参数优化完成')
print(f'最优参数: {optimization_result["best_params"]}')
print(f'最优值 (RMSE): {optimization_result["best_value"]:.6f}')

# 使用最优参数更新模型
predictor.update_model_parameters(optimization_result['best_params'])
print('✅ 模型参数已更新')

# 重新评估优化后的模型
optimized_metrics = predictor.evaluate(features, target)
print('
优化后模型指标:')
for metric, value in optimized_metrics.items():
    print(f'  {metric}: {value:.6f}')

## 7. 模型解释

使用SHAP分析因子重要性，监控市场趋势变化

In [ ]:
# 创建SHAP解释器
explainer = create_shap_explainer(predictor.model, features)
shap_values = calculate_shap_values(explainer, features)

print('✅ SHAP解释器创建完成')
print(f'SHAP值形状: {shap_values.shape}')

In [ ]:
# 分析因子重要性
factor_importance = analyze_factor_importance(shap_values, features)

print('📊 因子重要性排序:')
print(factor_importance)

# 可视化因子重要性
plt.figure(figsize=(10, 6))
plt.barh(factor_importance['feature'], factor_importance['importance'])
plt.xlabel('重要性')
plt.title('申万行业动量因子重要性')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 7.1 SHAP值可视化

In [ ]:
# 绘制SHAP摘要图
try:
    import shap
    shap.summary_plot(shap_values, features, feature_names=features.columns)
    plt.title('SHAP值摘要图')
    plt.tight_layout()
    plt.show()
except ImportError:
    print('⚠️  需要安装shap库以显示高级可视化')
    print('运行: pip install shap')

## 8. 行业分析

分析不同行业的动量因子表现

In [ ]:
# 按行业分析因子表现
industry_analysis = factor_data.groupby('stock_code').agg({
    'short_momentum': 'mean',
    'medium_momentum': 'mean',
    'long_momentum': 'mean',
    'return': 'mean'
}).reset_index()

# 重命名股票代码为行业名称
industry_map = {
    'IND001': '银行',
    'IND002': '保险',
    'IND003': '证券',
    'IND004': '房地产',
    'IND005': '建筑装饰'
}
industry_analysis['industry'] = industry_analysis['stock_code'].map(industry_map)

print('📈 各行业动量因子表现:')
print(industry_analysis.round(4))

# 可视化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 短期动量
axes[0,0].bar(industry_analysis['industry'], industry_analysis['short_momentum'])
axes[0,0].set_title('短期动量因子')
axes[0,0].tick_params(axis='x', rotation=45)

# 中期动量
axes[0,1].bar(industry_analysis['industry'], industry_analysis['medium_momentum'])
axes[0,1].set_title('中期动量因子')
axes[0,1].tick_params(axis='x', rotation=45)

# 长期动量
axes[1,0].bar(industry_analysis['industry'], industry_analysis['long_momentum'])
axes[1,0].set_title('长期动量因子')
axes[1,0].tick_params(axis='x', rotation=45)

# 平均收益率
axes[1,1].bar(industry_analysis['industry'], industry_analysis['return'])
axes[1,1].set_title('平均收益率')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 9. 结果总结

总结申万行业动量信号模型的分析结果

In [ ]:
print('🎯 申万行业动量信号模型分析总结')
print('=' * 50)

print('📊 数据概况:')
print(f'  数据记录数: {len(converted_data)}')
print(f'  行业数量: {len(industry_analysis)}')
print(f'  时间跨度: {converted_data["date"].min()} 到 {converted_data["date"].max()}')

print('\n📈 模型性能:')
print(f'  均方根误差 (RMSE): {evaluation_metrics.get("rmse", "N/A")}')
print(f'  信息比率 (IR): {evaluation_metrics.get("information_ratio", "N/A")}')

print('\n🔍 因子重要性:')
for idx, row in factor_importance.iterrows():
    print(f'  {row["feature"]}: {row["importance"]:.4f}')

print('\n🏭 行业表现:')
for idx, row in industry_analysis.iterrows():
    industry_name = industry_map.get(row['stock_code'], row['stock_code'])
    print(f'  {industry_name}: 平均收益率={row["return"]:.4f}, 短期动量={row["short_momentum"]:.4f}')

print('\n✅ 模型已完成以下任务:')
print('  • 申万行业数据加载与转换')
print('  • 动量因子计算与分析')
print('  • 随机森林模型训练')
print('  • 超参数优化')
print('  • 模型性能评估')
print('  • SHAP模型解释')
print('  • 行业间对比分析')

## 10. 使用真实数据

要使用真实申万行业数据，请按以下步骤操作:

In [ ]:
# 使用真实数据的示例代码
print('📋 使用真实申万行业数据步骤:')
print('1. 准备CSV文件，包含: date, industry_name, return 三列')
print('2. 文件格式示例:')
print('   date,industry_name,return')
print('   2020-01-01,银行,0.005')
print('   2020-01-01,保险,0.003')
print('   ...')
print('3. 修改上面代码中的 sw_data_path 变量指向你的数据文件')
print('4. 重新运行整个notebook')